In [26]:
from Bio import Entrez
from Bio import SeqIO

import re
import os

keyword="Synechococcus|cyanophage|Cyanophage"
accs=[]
titles=[]

acdict={}

splist=open("cyanophage_nuccore_result.txt").readlines()
for i in range(1,len(splist),4):
    if re.search(keyword, splist[i]):
        name=splist[i].split(",")[0].split(".")[1]
        name2=re.sub(r"complete genome\n|genomic sequence\n","", name)
        if name2 not in titles:
            titles.append(name2)
            acc=splist[i+2].split(" ")[0]
            accs.append(acc)
            acdict[acc]=name2
accs_str=",".join(accs)

Entrez.email="julia.brown@einstein.yu.edu"
for a in accs:
    handle=Entrez.efetch(db="nucleotide", id=str(a), rettype="gb", retmode="text")
    filename="./SynGenomes/"+a+".gbk"
    out=open(filename, "w")
    out.write(handle.read())
    out.close()
    handle.close()
    SeqIO.convert("./SynGenomes/"+a+".gbk","gb","./SynGenomes/"+a+".fasta","fasta")
    print(a+" is finished")



NC_016656.1 is finished
NC_020845.1 is finished
NC_009531.1 is finished
DQ875742.1 is finished
NC_016659.1 is finished
NC_016658.1 is finished
NC_016657.1 is finished
NC_019516.1 is finished
NC_022751.1 is finished
NC_013021.1 is finished
KP211958.1 is finished
DQ149023.2 is finished
NC_013085.1 is finished
NC_015569.1 is finished
NC_015289.1 is finished
NC_015287.1 is finished
NC_015286.1 is finished
NC_015282.1 is finished
NC_015281.1 is finished
NC_015279.1 is finished
FM207411.1 is finished
NC_008296.2 is finished
KJ410740.1 is finished
AF338467.2 is finished
NC_006820.1 is finished
NC_016766.1 is finished
NC_016164.1 is finished
NC_015465.1 is finished
NC_015463.1 is finished
NC_020854.1 is finished
NC_020872.1 is finished
NC_020875.1 is finished
NC_020847.1 is finished
NC_020836.1 is finished
NC_021072.1 is finished
NC_021071.1 is finished
NC_020865.1 is finished
NC_020857.1 is finished
NC_020855.1 is finished

tRNAscan-SE v.1.3.1 (January 2012) - scan sequences for transfer RNAs

In [30]:
from Bio.SeqUtils import GC

taxdict={}
sizedict={}
out=open("syn.genomeinfo.txt","w")

for phage in accs:
    gbk=SeqIO.parse("./SynGenomes/"+phage+".gbk", "genbank")
    for rec in gbk:
        taxonomy=rec.annotations["taxonomy"]
        if len(taxonomy)>4:
            tax=taxonomy[3]
        else:
            tax=taxonomy[-1]
        taxdict[phage]=tax
        sizedict[phage]=len(rec.seq)
        gc_content=GC(rec.seq)
        out.write(phage+"\t"+acdict[phage]+"\t"+tax+"\t"+str(len(rec.seq))+"\t"+str(gc_content)+"\n")
out.close()



In [ ]:
os.mkdir("./SynGenomes/syn_tRNA/")
out=open("syn_trna_scan.sh","w")
for phage in accs:
    out.write("tRNAscan-SE -o ./SynGenomes/syn_tRNA/"+phage+"trnas.txt -G -D ./SynGenomes/"+phage+".fasta\n")
out.close()
!bash syn_trna_scan.sh

In [27]:
import glob
import os

names=glob.glob("./SynGenomes/syn_tRNA/*")

tRNA_phages=[]

for n in accs:
    tfile="./SynGenomes/syn_tRNA/"+n+"trnas.txt"
    if os.path.getsize(tfile)>0:
        #print("phage "+phage+" has tRNAs")
        tRNA_phages.append(n)
    #else:
     #   print("phage "+phage+" does not have tRNAs")

print("there are a total of "+str(len(names))+" phages but only "+str(len(tRNA_phages))+" have tRNAs")

out=open("syn_tRNA_summary.txt","w")
out.write("phage_short\tphage_full\tbound1\tbound2\ttRNA\tanti_codon\tintron_bound1\titron_bound2\tscore\n")

for phage in tRNA_phages:
    info=open("./SynGenomes/syn_tRNA/"+phage+"trnas.txt").readlines()
    for line in info[3:]:
        vec=line.split("\t")
        
        out.write(acdict[phage]+"\t"+vec[0]+"\t"+vec[2]+"\t"+vec[3]+"\t"+vec[4]+"\t"+vec[5]+"\t"+vec[6]+"\t"+vec[7]+"\t"+vec[8])
out.close()

there are a total of 39 phages but only 22 have tRNAs


In [28]:
%load_ext rpy2.ipython
%R library(dplyr);library(magrittr); library(ggplot2)

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


<StrVector - Python:0x10b8ec950 / R:0x101779208>
[str, str, str, ..., str, str, str]

In [20]:
%%R 
trna<-read.table("syn_tRNA_summary.txt", sep="\t", header=TRUE)
trna%>%group_by(phage_short)%>%summarise(count=n())%>%.[order(-.$count),]

Source: local data frame [23 x 2]

                    phage_short count
1   Synechococcus phage S-CRM01    32
2     Synechococcus phage S-PM2    25
3    Synechococcus phage S-RSM4    12
4   Synechococcus phage S-RSM4     12
5     Synechococcus phage S-SM2    11
6             Cyanophage S-TIM5    10
7          Cyanophage KBS-M-1A      8
8             Cyanophage Syn10      6
9              Cyanophage Syn2      6
10            Cyanophage Syn30      6
..                          ...   ...
